## Homework 3 - Places of the world

In [2]:
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import requests
import os
import pandas as pd
import numpy as np
import time
import pickle
import re
import csv 

### 1. Data collection
#### 1.1 Get the list of places

**Collectings the URLs** of the places listed in **the first 400 pages** of **[Atlas Obscura](https://www.atlasobscura.com/)**

In [53]:
urls = []           # list to store URLs
tot_pages = 400     # number of pages requested
page = 1            # set counter

places_file  = open("places_urls.txt", "w+")  # open empty .txt file to write URLs

with tqdm( total = tot_pages, position=0, leave=True) as pbar:
    
    while page < (tot_pages+1):
        url = f"https://www.atlasobscura.com/places?page={page}&sort=likes_count"
        result = requests.get(url, headers = {'User-agent': 'your bot 1.1'})    # make a request to web pages and retrive HTML code
        soup = bs(result.text, 'html.parser')                                   # parsing HTML with beautifulsoup
        
        if result.status_code == requests.codes.ok:          # check the response status code
        
            for a in soup.find_all('a',{"class":"content-card content-card-place"}, href=True):
                places_file.write(a['href'] +'\n')           # store URLs in .txt file
                urls.append(a['href'])                       # store URLs in list
                
            pbar.update()
            page = page+1
            
        else:
            time.sleep(200)
            
print(" URLs found :", len(urls), " \t unique URLs :", len(set(urls)))


100%|██████████| 400/400 [05:37<00:00,  1.19it/s]  

 URLs found : 7200  	 unique URLs : 7200


#### 1.2 Crawl places

**Downloading HTML code** of each place from urls list and saving it as **.txt file** in folder (folder corresponding to Atlas Obscura page number of the place):

In [77]:

c = 1 # # set folder counter

for i in tqdm(range(400)):
    newpath = r'./places/' + str(i+1)       # create one folder for each iteration, each folder will contain 18 places as .txt documents
    os.makedirs(newpath, exist_ok = True)
    
    for u in urls[18*i:18*(i+1)]: 
        result = requests.get( url = "https://www.atlasobscura.com" + u, 
                              headers = {'User-agent': 'your bot 1.1'})   # make a request to web pages and retrive HTML code
        
        if result.status_code == requests.codes.ok:                      # check the response status code
            name = newpath +'/document_' + str(c) + '.txt'              # name directory for the new file 
        
            with open(name, "w") as file:
                file.write(result.text)       # save HTML page content as .txt file
            
            time.sleep(0.5)
            c += 1
        else:
            time.sleep(200)
            

100%|██████████| 400/400 [00:00<00:00, 791005.00it/s]


#### 1.3 Parse downloaded pages

Collecting the places's information we desire with BeautifulSoup:

In [16]:
##

def extract_single_place( page ):
    ''' 
    Function to parse HTML content with BeautifulSoup and retrive document data :
        Argument: .txt file 
        Returns: dictionary with place information
    ''' 
    soup = bs(page, 'html.parser')
    
    title = soup.find_all("h1", {"class": "DDPage__header-title"})[0].contents[0].text                                                  #1                  
    
    try:
        tag = soup.find_all("div", {"class": "item-tags col-xs-12"})[0]
        place_tag = [i.contents[0].text.replace("\n","") for i in tag.find_all("a")]                                                    #2
    except IndexError:
        place_tag = "" 
        
    short_desc = soup.find_all("h3", {"class": "DDPage__header-dek"})[0].contents[0].text                                               #3
    numPeopleVisited = soup.find_all("div", {"class": "title-md item-action-count"})[0].contents[0].text                                #4
    numPeopleWant = soup.find_all("div", {"class": "title-md item-action-count"})[1].contents[0].text                                   #5
    placeDesc = soup.find_all("div", {"id": "place-body"})[0].text                                                                      #6
    placeAddress = str(soup.find_all("aside", {"class": "DDPageSiderail__details"})[0].contents[1].text).replace("\n","")               #7
    placeAlt_placeLong = str(soup.find_all("aside", {"class": "DDPageSiderail__details"})[0].contents[3].text).replace("\n","")         #8
    l = placeAlt_placeLong.split(",")
    
    try:    
        placePubDate = soup.find_all("div", {"class": "DDPContributor__name"})[0].text                                                   #9
    except IndexError:
        placePubDate = ""
        
    try:
        my_div0 = soup.find_all("div", {"class": "card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})[0]          #10
        placeNearby = [i.contents[1].text for i in my_div0.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})]
    except IndexError:
        placeNearby = ""
    
    try:
        placeEditors = soup.find_all("a", {"class": "DDPContributorsList__contributor"}, href=True)                                        #11
        placeEditors = [x.text.replace("\n","") for x in placeEditors]
    except IndexError:
        placeEditors = ""
        
    try:
        my_div = soup.find_all("div", {"class": "card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})[1]             #12
        placeRelatedPlaces = [i.contents[1].text for i in my_div.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})]
    except IndexError:
        placeRelatedPlaces = ""
        
    try:
        my_div2 = soup.find_all("div", {"class": "card-grid CardRecircSection__card-grid js-inject-gtm-data-in-child-links"})[2]
        placeRelatedLists = [i.contents[1].text for i in my_div2.find_all("h3", {"class":"Card__heading --content-card-v2-title js-title-content"})]  #13
    except IndexError:
        placeRelatedLists = ""
        
    #page_url = "https://www.atlasobscura.com" + url

    data = {"title" : title, "place_tag": place_tag, "numPeopleVisited" : numPeopleVisited, "numPeopleWant": numPeopleWant, "placeDesc": placeDesc, 
            "short_desc": short_desc, "placeNearby":placeNearby, "placeAddress":placeAddress,"placeAlt": l[0], "placeLong":l[1],
           "placeEditors":placeEditors, "placePubDate":placePubDate, "placeRelatedPlaces": placeRelatedPlaces, "placeRelatedLists":placeRelatedLists}
        
    return data


In [51]:
file = open('./places_urls.txt', 'r')
urls = file.read().splitlines()
file.close()

newpath = "./Posti"
os.makedirs(newpath, exist_ok = True)

for i in tqdm(range(400)):
    
    for j in range(18):
        
        path = './places/'+str(i+1)+'/document_'+str(18*i+j+1)+'.txt'  #find documents path in places directory 
        my_file = open(path)                            # open .txt file → it is place HTML code 
        my_dict = extract_single_place(my_file)         # applay BeautifulSoup function to get the information we desire for each place
        my_dict['placeURL']=urls[18*i+j]                # add URLs to dictionary of place information
        
        name = newpath + '/document_'+str(18*i+j+1)+'.tsv'         # file name with document_id
       
        with open(name, 'w') as f:  # You will need 'wb' mode in Python 2.x
            w = csv.DictWriter(f, delimiter='\t', fieldnames = list(my_dict.keys()))     # save as tsv file
            #w.writeheader()
            w.writerow(my_dict)


100%|██████████| 400/400 [27:14<00:00,  4.09s/it]


It is usefull to create a Pandas dataframe containing all the information from every .tsv files :

In [75]:
# merge .tsv files in Pandas dataframe :


colonne = ["placeName", "placeTags", "numPeopleVisited","numPeopleWant","placeDesc", "placeShortDesc", "placeNearby","placeAddress","placeAlt", 
           "placeLong", "placeEditors","placePubDate", "placeRelatedPlaces","placeRelatedLists","placeUrl"]

directory = 'Posti/'
data = []

for file in tqdm(os.listdir(directory)):
    #print(file)
    df = pd.read_csv(directory + file, sep= '\t')  
    data.append(df)
    
df.columns = colonne
data = pd.concat(data)
print(data.shape)
data = data.reset_index(drop=True)
data.to_csv("all_places_data.tsv", sep='\t', encoding='utf-8', index= False)

data.info()

100%|██████████| 7200/7200 [00:18<00:00, 380.03it/s]


(7200, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7200 entries, 0 to 7199
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   placeName           7200 non-null   object 
 1   placeTags           7166 non-null   object 
 2   numPeopleVisited    7200 non-null   int64  
 3   numPeopleWant       7200 non-null   int64  
 4   placeDesc           7200 non-null   object 
 5   placeShortDesc      7200 non-null   object 
 6   placeNearby         7199 non-null   object 
 7   placeAddress        7200 non-null   object 
 8   placeAlt            7200 non-null   float64
 9   placeLong           7200 non-null   float64
 10  placeEditors        7200 non-null   object 
 11  placePubDate        7199 non-null   object 
 12  placeRelatedPlaces  7171 non-null   object 
 13  placeRelatedLists   2765 non-null   object 
 14  placeUrl            7199 non-null   object 
dtypes: float64(2), int64(2), object(11)
memory u

### 2. Search Engine

Now **pre-processing** the **place description column** of places dataframe.

In [3]:
import string
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem.snowball import SnowballStemmer
snowstem = SnowballStemmer('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We write a function to remove **punctuation**, **stop-words**, get **stem words** and delete **special character**

In [4]:
def processing_test(stringa : str):
    
    ''' 
    Function to pre-process strings :
        Argument: string, it's the description of each place
        Returns: list of tokenized and processed words  
    '''
    
    stringa = stringa.replace("\n","").replace("\xa0","").lower()                         # remove unwanted characters
    stringa = stringa.translate(str.maketrans('', '', string.punctuation))               # remove punctuation with "string" module
    stringa = [ token for token in stringa.split(' ') if token not in stop_words ]      # remove stop-words with nltk
    output = [snowstem.stem(word) for word in stringa]                                 # stemming string with nltk
    
    
    return output

In [5]:
# apply processing_test function to data.placeDesc column
data = pd.read_csv('all_places_data.tsv', sep ='\t')
data['list_words'] = data.placeDesc.apply(lambda desc: processing_test(desc))  ## add new data.list_words column with tokenized and processed words

#### 2.1 Conjunctive query

##### 2.1.1 Create your index!

In [5]:
from collections import Counter 
from functools import reduce

In [60]:
vocabulary = Counter(reduce(lambda x,y: x+y, data.list_words))

# Dictionary : 
#     key = unique words that appear in place descriptions
#     value = word frequency counter in all corpus / dataset

print(" Unique tokenized words in corpus of documents : ", len(vocabulary.keys()))

 Unique tokenized words in corpus of documents :  89094


In [61]:
# Create a file named "vocabulary" that maps each word to an integer (term_id) :

word_dict = {}      # initialize empty dictionary
item_id = 1         # set term_id counter

for word in vocabulary.keys():  # iterating over unique words in corpus of documents
    word_dict[word] = item_id   # assign an integer = term_id to each word 
    item_id += 1
    
#word_dict.keys()

z = open("vocabulary.pkl", "wb")  
pickle.dump(word_dict, z)          # with pickle module save word_dict as .pkl file named "vocabulary"
z.close()

In [6]:
# load  word_dict as dictionary if needed : 

with open('vocabulary.pkl', 'rb') as f:
    word_dict = pickle.load(f)

In [ ]:
# Building the Inverted Index as dictionary :

inverted_idx = {}   

# Dictionary : 
#     key = term_id (integer) from word_dict.values
#     value = document_id (integer) that contain word corresponding to term_id 
#              (document_id = row index of our dataFrame which contains it in list_words column)

for word, item_id in tqdm(word_dict.items()):
    inverted_idx[item_id] = list(data[data.list_words.apply(lambda row: word in row)].index)
    
    
II = open("II.pkl", "wb")
pickle.dump(inverted_idx, II)    # with pickle module save inverted_idx as .pkl file 
II.close()

In [7]:
# load inverted_idx as dictionary if needed : 

with open('II.pkl', 'rb') as f:
    inverted_idx = pickle.load(f)

Let's try an example:

In [8]:
# looking for word "food" and retrive document_id of documents that contains it (in description) :

text = ["food"]
example = [set(inverted_idx[item]) for item in [word_dict[word] for word in text]][0]   # indexes of all the document the contain the word 'food'

print(" looking for word :", *text,"\tfound in {} documents".format(len(example)))
print("\n indexes : ", list(example))

display(data.loc[list(example)][["placeName", "placeDesc", "placeUrl"]].head(3))

 looking for word : food 	found in 355 documents

 indexes :  [6150, 8, 4109, 2063, 4112, 6163, 22, 2071, 6169, 31, 6179, 6188, 4144, 6194, 2099, 52, 4151, 4155, 6211, 4167, 4181, 4186, 6236, 4190, 4192, 98, 101, 2155, 2159, 4208, 6264, 134, 139, 4235, 156, 6302, 6303, 4257, 4258, 6310, 168, 179, 4279, 2237, 2238, 2241, 4292, 2252, 2253, 207, 4316, 242, 246, 249, 6399, 6402, 4359, 271, 275, 2336, 2338, 2339, 2352, 305, 2353, 2366, 4419, 4421, 6476, 2381, 2386, 2406, 6503, 4461, 371, 6524, 2434, 389, 399, 405, 4503, 4504, 2469, 4520, 4526, 4529, 4532, 2489, 6588, 457, 6605, 6607, 4560, 6610, 2522, 4576, 4578, 6627, 4589, 2542, 496, 4597, 2550, 4598, 4608, 6659, 2580, 6677, 538, 6686, 4642, 4651, 4677, 6731, 591, 600, 6744, 2662, 2669, 624, 2685, 2691, 6787, 2715, 2717, 671, 4767, 673, 4779, 2748, 705, 4803, 4805, 4807, 717, 6863, 729, 6896, 6897, 768, 6912, 6920, 6922, 4892, 2846, 2853, 4902, 2874, 2875, 6971, 2878, 4926, 6975, 6984, 2892, 4957, 2914, 869, 7016, 4973, 4977, 2946, 7046, 

,placeName,placeDesc,placeUrl
6150,Salt Ponds of San Francisco Bay,As the plane banks to make its final pass befo...,https://www.atlasobscura.com/places/salt-ponds...
8,Tchai-Ovna House of Tea,"Built as recently as 2002, the Tchai-Ovna is a...",https://www.atlasobscura.com/places/tchaiovna
4109,Shopsin's,"Kenny Shopsin’s philosophy is strict, but simp...",https://www.atlasobscura.com/places/shopsins


##### 2.1.2 Execute the query

**Building** first **search engine** as function that executes queries and shows **placeName**, **placeDesc** and **placeURL** of results :

In [8]:
def simple_search(query_str : str, dataframe):
    
    ''' 
    Function to retrive documents that contain all the terms of the query :
        Argument: string, it's the user input query (words considered as conjunctive query-AND)
        Returns: pandas Dataframe with placeName, placeDesc and placeURL of Atlas Obscura's place that match input string
    '''

    if len(query_str) == 0:
        return print("\t empty query")
    else:
        try:
            query = processing_test(query_str)   # tokenize query
            
            result = [set(inverted_idx[item]) for item in [word_dict[word] for word in query]]
            result = set.intersection(*map(set,result))        # look for intersection between indexes of documents that contain one of query's words
            
            output = dataframe.loc[list(result)][["placeName", "placeDesc", "placeUrl"]]  # match inverted indexes with documents pd.Dataframe.index 
            print("\n results : \t")
            display(output.head())
            print(" data.shape :",output.shape)
            your_list = list(result)
        except KeyError:                            # take care of queries not found
            return print("Error 404: no documents found that match query")
    return (output, your_list)

In [10]:
query = input(str("\t query :"))
result = simple_search( query, data )


 results : 	


,placeName,placeDesc,placeUrl
1538,"Carnegie Library of Washington, D.C.",In 1899 a member of the D.C. Library board une...,https://www.atlasobscura.com/places/carnegie-l...
3593,Hall of Heroes,Most superheroes tend to make their base in a ...,https://www.atlasobscura.com/places/hall-of-he...
6669,Chapultepec Castle,When we think of castles we think of Europe an...,https://www.atlasobscura.com/places/chapultepe...
528,Museum of the Eye,How do you see the world? Find out at the Muse...,https://www.atlasobscura.com/places/museum-oph...
3600,McNutt Sculpture Garden,A hidden sculpture garden exists just off the ...,https://www.atlasobscura.com/places/mcnutt-scu...


 data.shape : (234, 3)


#### 2.2 Conjunctive query & Ranking score

The next step is to make a new Search engine that will that into consideration the similarity that the documents have with the query.
So we want to find all the documents that contain all the words in the query and, after computing the TF-IDFs, to sort them by their similarity with the query.

In [13]:
clean_desc = {}
for i in tqdm(range(len(data))):
    # get the description from the data frame, clean it and save it in a dictionary
    clean_desc[i]=' '.join(data.list_words[i])
    
# We save dictionaries as pkl
descriptionsDataset = open("descriptionsDataset.pkl", "wb")
pickle.dump(clean_desc, descriptionsDataset)
descriptionsDataset.close()

100%|██████████| 7200/7200 [00:00<00:00, 78470.52it/s]


In [9]:
# To get the descriptions
with open('descriptionsDataset.pkl', 'rb') as f:
    clean_desc = pickle.load(f)

Let's start with creating a new Inverted Indexes dictionary, but this time we will save also the TF-IDFs

In [12]:
# Function that will compute the tf-idf given a document and a term
def compute_tfidf(i, desc, d_i, D):
    # Count how many times the term i is un the document
    occurence = 0
    for w in desc:
        if w == i:
            occurence += 1
    # tf = how many times the term i is un the document/the total number of words in the document
    tf = occurence/len(desc)
    # idf indicate how 'specific' is the term i = log(number of documents/number of documents that contains i)
    idf = np.log2(D/d_i)
    return tf*idf

In [18]:
# create the actual dictionary
InvertedIdx_v2 = {}
for i in inverted_idx.keys():
    #how many documents contain the term i
    d_i = len(inverted_idx[i])  
    for j in range(d_i):
        # get the full (cleaned) description of the document InvertedIdx[i][j]
        desc = clean_desc[inverted_idx[i][j]] 
        #compute the tfidf
        tfidf = compute_tfidf(i, desc.split(), d_i, 7200)
        if i not in InvertedIdx_v2.keys():
            InvertedIdx_v2[i] = [(inverted_idx[i][j], tfidf)]
        else:
            InvertedIdx_v2[i].append((inverted_idx[i][j], tfidf))
       

Again we can save it:

In [19]:
II_v2 = open("tfidfII.pkl", "wb")
pickle.dump(InvertedIdx_v2, II_v2)
II_v2.close()

And access it in a easier way, now that it's saved:

In [10]:
with open("tfidfII.pkl", 'rb') as f:
    InvertedIdx_v2 = pickle.load(f)

To compute the similarity (cosine similarity) between the documents we are goint to need some other informations:
- Have the IDFs of the terms, so we can compute the query vector
- The norms of all the documents

Just like the Inverted Indexes we will compute all the informations needed and save them externally

In [21]:
IDFs = {}
for i in inverted_idx.keys():
    d_i = len(inverted_idx[i])  #how many documents contain the term i
    IDFs[i] = np.log10(7200/d_i)
# Save the IDFs externally
idfs = open("SingleIDFs.pkl", "wb")
pickle.dump(IDFs, idfs)
idfs.close()

In [11]:
# To load the IDFs
with open("SingleIDFs.pkl", 'rb') as f:
    IDFs = pickle.load(f)

In [25]:
# compute the norm of the documunt doc using the TF-IDFs saved in the Inverted Indexes
def doc_norm(terms, InvertedIdx_v2, doc):
    sums = []
    # to keep track of the already checked terms
    checked = set()
    for w in terms:
        # If w is not checked
        if w not in checked:
            checked.add(w)
            for j in InvertedIdx_v2[word_dict[w]]:
                if j[0] == doc:
                    # add to the list the square of the TF_IDF of the term w
                    sums.append((j[1])**2)
                    break
    # return the norm = the square root of the sum of all the TF_IDF^2 
    return np.sqrt(sum(sums))
    

In [29]:
norms = {}
# for all the documents
for i in clean_desc.keys():
    # Compute the norm
    norm = doc_norm(clean_desc[i].split(), InvertedIdx_v2, i)
    # save it in the dictionary 
    norms['document_'+str(i+1)]=norm
# Save the dictionary externally
Norm = open("norms.pkl", "wb")
pickle.dump(norms, Norm)
Norm.close()

In [21]:
# To load the norms
with open("norms.pkl", 'rb') as f:
    norms = pickle.load(f)

Now that we saved everything that we need it's time for the actual search engine, that's gonna be implemented using the cosine similarity

In [12]:
# function that given a word and the inverted indexes dictionary, gives back the list that correspond to the word as a df row
def df_linew (word, II):
    documents = []
    tfidf = []
    # takes all the touples in the list
    for d in II[word_dict[word]]:
        # Gets the index of the documents
        documents.append('document_'+str(int(d[0])+1))
        # and all the tf_idf 
        tfidf.append(d[1])
    # put the documents as col names and the tf_idf as values of the df
    df = pd.DataFrame.from_dict({word: tfidf}, orient='index', columns=documents)
    return df

In [13]:
# Function that given a query and the dictionary of the Inverted Indexes and gives back a df 
# df with columns that correspond to the terms of the query and row to the deocuments that contains the terms
def searchEngine(query, InvertedIdx_v2):  
    Engine = pd.DataFrame()
    # for all the terms of the query
    for w in query:
        # make the df row of the term w
        df_idf = df_linew(w, InvertedIdx_v2)
        # concat the new row
        Engine = pd.concat([Engine, df_idf]).fillna(0)
    Engine = Engine.transpose()
    return Engine.loc[(Engine!=0).all(axis=1)]

In [14]:
# compute the tfidf of the query
def queryTFIDF(query, IDFs):
    q = []
    for i in range(len(query)):
        q.append(IDFs[word_dict[query[i]]]/len(query))
    return q


In [15]:
# Function that compute the cosine similarity, will return a list of touples as (doc_it, cosine_sim)
def cosineDistance(Search, query, IDFs, norms):
    d = []
    # create the query vector with the TF-IDFs inside
    q = queryTFIDF(query, IDFs)
    # for all the documents that contain all the query terms
    for i in Search.index:
        aux = np.dot(list(Search.loc[i]),q)
        # append the touples
        d.append((i,aux/(norms[i])))
    return d

In [16]:
# Function that print the results of the search engine
def print_search(result, place_df, query):
    print('While looking for the query:\n\t', query)
    k = int(input())
    print('Those where the top', k, 'results:')
    wantedCols = ['placeName', 'placeDesc', 'placeURL']
    indexes = []
    simil = []
    for i in range(len(result)):
        indexes.append(int(result[i][0].split('_')[1])-1)
        simil.append(result[i][1])
    # load the df
    df = place_df.iloc[indexes][wantedCols]
    # add the last column
    df['Similarity']=simil
    display(df.head(k))


In [ ]:
query = input()
query1 = processing_test(query)
Search = searchEngine(query1, InvertedIdx_v2)
result = mergesort(cosineDistance(Search, query1, IDFs, norms))
print_search(result, data, query)

While looking for the query:
	 american museum
Those where the top 10 results:


,placeName,placeDesc,placeURL,Similarity
6206,Sweet Home Cafe,Thomas Downing was the oyster king. In 19th-ce...,https://www.atlasobscura.com/places/sweet-home...,0.143841
140,Museum of the Weird,The dime or dime store museum is by all accoun...,https://www.atlasobscura.com/places/museum-weird,0.123250
1201,Harvard Museum of Natural History,Collecting three different institutions into o...,https://www.atlasobscura.com/places/harvard-mu...,0.122421
6283,Self-Taught Genius Gallery,"In 2017, the American Folk Art Museum in Manha...",https://www.atlasobscura.com/places/self-taugh...,0.109571
6220,Siriraj Medical Museum,The Siriraj Medical Museum abounds with medica...,https://www.atlasobscura.com/places/siriraj-me...,0.107924
4463,American Writers Museum,The American Writers Museum—tucked away on the...,https://www.atlasobscura.com/places/american-w...,0.101455
1899,National World War II Museum,"Perhaps once thought too narrowly focused, thi...",https://www.atlasobscura.com/places/national-w...,0.100796
4987,Mammoth Cave Wildlife Museum,Cave City is a wonderland of kitschy tourist t...,https://www.atlasobscura.com/places/mammoth-ca...,0.099418
2756,Museum of Mourning Art,Mourning and personal response to death are un...,https://www.atlasobscura.com/places/museum-of-...,0.097444
6981,National Museum of the Pacific War,Dedicated specifically to remembering the stor...,https://www.atlasobscura.com/places/national-m...,0.095261


# 3

Now we want to try and create a new score to sort the result of our query. 

Up until now we were thinking of all this places just as documents to sort by looking at the description. Now what we want to do and sort the documents using the distance from a given point (latitude, longitude). 

So given a query of terms and a location, the result will be given by the result of the query sorted by the distance.

In [9]:
import plotly.express as px
import geopy.distance

In [10]:
# function that compute the new score for the new 'similarity', the distance our case
def NewScore1(lat, long, results, place_df):
    NewResult = []
    for i in results:
        # get the coordinates of the places
        lat_i = place_df.iloc[i]['placeAlt']
        long_i = place_df.iloc[i]['placeLong']
        # compute the distance from the coordinates given as imput
        dist = (geopy.distance.geodesic((lat,long), (lat_i,long_i)).km)
        # add the touple (doc_id, distance) to a list
        NewResult.append((i,-dist))

    return NewResult


In [11]:
# Function that show the results from the new search engine
def ShowNewScore1(result, place_df, query, lat, long, k):
    print('While looking for the query:\n\t', query, '\nIn the position:\n\t', lat, long,'\nThose were the top', k, 'results:')
    wantedCols = ['placeName', 'placeDesc','placeAlt', 'placeLong', 'placeUrl', 'numPeopleVisited', 'numPeopleWant','placeAddress']
    indexes = []
    dist = []
    for i in range(len(result)):
        indexes.append(result[i][0])
        dist.append(str(round(-result[i][1],3))+' km')
    # load the df
    df = place_df.iloc[indexes][wantedCols]
    # add the last column
    df['Distance']=dist
    display(df[['placeName','placeDesc', 'placeUrl', 'Distance']].head(k))
    return df

In [13]:
# Get the query
print('Write the term query for the search:')
query = input()

# Get the lat and long
print('Write the location (latitude and longitude separeted by a space):')
lat, long = map(float, input().split())

# Get how many places to display
k = int(input())

# find the list of the indexes of the documents that contain all the terms of the query
qSearch = simple_search(query, data)

#let's look for the closer places
fullSearch = NewScore1(lat, long, qSearch[1], data)
fullSearchSorted = mergesort(fullSearch)

newdf = ShowNewScore1(fullSearchSorted, data, query, lat, long, k)

Write the term query for the search:
Write the location (latitude and longitude separeted by a space):

 results : 	


,placeName,placeDesc,placeUrl
1538,"Carnegie Library of Washington, D.C.",In 1899 a member of the D.C. Library board une...,https://www.atlasobscura.com/places/carnegie-l...
3593,Hall of Heroes,Most superheroes tend to make their base in a ...,https://www.atlasobscura.com/places/hall-of-he...
6669,Chapultepec Castle,When we think of castles we think of Europe an...,https://www.atlasobscura.com/places/chapultepe...
528,Museum of the Eye,How do you see the world? Find out at the Muse...,https://www.atlasobscura.com/places/museum-oph...
3600,McNutt Sculpture Garden,A hidden sculpture garden exists just off the ...,https://www.atlasobscura.com/places/mcnutt-scu...


 data.shape : (234, 3)
While looking for the query:
	 american museum 
In the position:
	 40.7512 -73.9769 
Those were the top 10 results:


,placeName,placeDesc,placeUrl,Distance
1020,The American Kennel Club Museum of the Dog,At the intersection of the Venn diagram where ...,https://www.atlasobscura.com/places/the-americ...,0.101 km
545,Theodore Roosevelt Birthplace Museum,Behind an otherwise innocuous (if immaculately...,https://www.atlasobscura.com/places/theodore-r...,1.71 km
418,Albertine,A hand-painted ceiling of celestial scenes cap...,https://www.atlasobscura.com/places/albertine,3.033 km
1316,Self-Taught Genius Gallery,"In 2017, the American Folk Art Museum in Manha...",https://www.atlasobscura.com/places/self-taugh...,3.848 km
3997,Museum of Chinese in America,The Museum of Chinese in America is nestled—al...,https://www.atlasobscura.com/places/museum-of-...,3.998 km
2241,Museum of Food and Drink,It’s easy to work up an appetite as you meande...,https://www.atlasobscura.com/places/museum-of-...,4.274 km
3793,The Oldest Fence in New York,Most visitors to Bowling Green today come to s...,https://www.atlasobscura.com/places/the-oldest...,6.008 km
6446,Fraunces Tavern,Fraunces Tavern was originally the home of ear...,https://www.atlasobscura.com/places/fraunces-t...,6.056 km
1058,Governors Island,Governors Island has quite a lot of history to...,https://www.atlasobscura.com/places/governor-s...,7.532 km
2194,Hamilton Grange,Built in the pastoral expanses of colonial Har...,https://www.atlasobscura.com/places/hamilton-g...,8.212 km


# 4

It may be usefull to also use a visual raffiguration of our results. So here there will be a map of the results of the previous search engine: A world map centred in the coordinated given in input where the results are bigger the higher is the popolarity of the place (average of people that want to go and the people that actually visited the place).

In [14]:
average = [sum(x) for x in zip(list(newdf.numPeopleWant),list(newdf.numPeopleVisited))]
average[:] = [x/2 for x in average]
newdf['Popolarity'] = average

In [15]:

px.set_mapbox_access_token(open(".mapbox_token").read())
# produce a map with the results from point 3
fig = px.scatter_mapbox(newdf, lat=newdf["placeAlt"], lon=newdf["placeLong"], color="Distance", center=dict({'lat':lat, 'lon':long}),
hover_name = newdf['placeName'], hover_data=["placeAddress", "numPeopleVisited"], size="Popolarity", zoom=8)
fig.show()


### 6. Command line question 

To solve this task we considered a smaller dataset with just the columns we needed (numPeopleVisited, numPeopleWant and placeAddress).\
We used this dataset as input for our **CommandLine.sh** file

In [70]:
new_df = data [["numPeopleVisited", "numPeopleWant","placeAddress"]]
new_df.to_csv('_places_data_.csv', encoding="utf-8", index=False, sep="\t")
new_df.head(4)

,numPeopleVisited,numPeopleWant,placeAddress
0,1190,595,Prater 9 Vienna Austria
1,349,871,"Piazza di San Silvestro, 17 Rome, 00187 Italy"
2,331,693,"529 North Charles Street Baltimore, Maryland, ..."
3,1136,2170,"514 Chartres Street New Orleans, Louisiana, 70..."


In [71]:
%%sh
sh CommandLine.sh

How many places can be found in Italy? 210 places
How many people, on average, have visited the places in Italy? 385.443 people
How many people in total want to visit the places in Italy? 183331 people

How many places can be found in Spain? 91 places
How many people, on average, have visited the places in Spain? 451.582 people
How many people in total want to visit the places in Spain? 71714 people

How many places can be found in France? 206 places
How many people, on average, have visited the places in France? 427.772 people
How many people in total want to visit the places in France? 205732 people

How many places can be found in England? 366 places
How many people, on average, have visited the places in England? 477.795 people
How many people in total want to visit the places in England? 390142 people

How many places can be found in United States? 4615 places
How many people, on average, have visited the places in United States? 438.978 people
How many people in total want to vis

**cut** = command to select parts of lines from a file\
**grep** = command to search for pattern (regular expression) in FILE, used to distinguish the country of places\
**aws** = to sum and compute average over specific column

In [72]:
cat CommandLine.sh

#!/bin/sh

array=("Italy" "Spain" "France" "England" "United States")           # array of countries

for val in "${array[@]}"; do                                         # interating in array
     
    output=$(cut -f3 _places_data_.csv | grep "$val" | wc -l)        # grep country from 3rd columns of csv and do word count
    echo "How many places can be found in $val? $output places"      # print result
    
    output2=$(cut -f1,3 all_places_data.csv | grep "$val"| awk -F"," '{sum+=$1} END {print sum/NR}') 
    
                                             # grep country then sum over the 1st column (numPeopleVisited) divided by len of records
            
    echo "How many people, on average, have visited the places in $val? $output2 people"          
    
    output3=$(cut -f2,3 all_places_data.csv | grep "$val"| awk -F"," '{sum+=$2} END {print sum}')   # grep country and sum over the 2nd column (numPeopleWant)
    echo -e "How many people in total want to visit the places in $va

# 7

In [ ]:
import random

Define 3 sorting algorithms for the probleme of the ApplicantsInfo.txt, must be given as input the number of people that will apply and their grades, then it will give back the sorted result over their average.

### mergesort algorithm

In [12]:
# Sorting algorithm mergesort 

# Function that check the max between x and y
def ismore(x, y):
    # The average of y is bigger: so I return false (0)
    if x[1] < y[1]: return 0
    # The average of y and x is equal, but the name of y comes first in the alphabeth: so I return false
    if x[1] == y[1] and x[0] > y[0]: return 0
    # If i get to this point x is the max
    return 1

# Function that merges together two already sorted lists in only one new sorted list 
def merge(a1, a2):
    i = 0 
    j = 0 
    A = []
    # Goes through both lists at the same time and compare the elements, untill I get to the end of one of the two lists
    while i<len(a1) and j<len(a2):
        # In each iteration append the element that rapresent the highest average
        if ismore(a1[i], a2[j]):
            A.append(a1[i])
            i +=1
        else:
            A.append(a2[j])
            j += 1

    # Append the elements still in a1, if there are any       
    while i<len(a1):
        A.append(a1[i])
        i += 1
    # Append the elements still in a2, if there are any  
    while j<len(a2):
        A.append(a2[j])
        j += 1
    return A


# average is going to be a list of touples that have at the first element the full name and in the second element the average grade 
def mergesort(average):
    if len(average) == 1: return average
    # Get the middle index (element)
    n = len(average)//2
    # Get the first n elements
    a1 = average[0:n]
    # Sort them 
    a1 = mergesort(a1)
    # Get the second n elements
    a2 = average[n:]
    # Sort them
    a2 = mergesort(a2)
    # Put everything together with the merge function
    return merge(a1, a2)


#### mergesort computational cost


The $mergesort$ algorithm here implemented is composed of two main functions:
- $mergesort$, with cost $T(n)$
- $merge$, with cost $T'(n,m)$

The $margesort$ functions divides the list (of length $n$) given in input in two lists (of length $\frac{n}{2}$), then calls itself on both the smaller lists, to sort them, and to finish put together the two lists of length $\frac{n}{2}$ with the $merge$ function.
$$T(n)=2\cdot T(\frac{n}{2}) + T'(\frac{n}{2},\frac{n}{2})$$

The $merge$ function get as input two sorted lists $l_1$ and $l_2$, of length $n$ and $m$, then goes throught both of them by: 
1. comparing their elements ${l_1}[i]$ and ${l_2}[j]$
2. adding the max between the elements to a new list $A$
3. increasing either $i$ or $j$ by one, depending to which I added to $A$

After either $i$ gets to $n$ or $j$ to $m$, so all the elements of one of the lists are in $A$, the $merge$ function append to $A$ the rest of the other list.
So $merge$ actually just does some costants operation (findind the max and appending) to all the elements of both lists and that implies:
$$ T'(n,m) = \mathcal{O}(n+m) $$

So we can conclude that: 
$$ T(n)=2\cdot T(\frac{n}{2}) + T'(\frac{n}{2},\frac{n}{2}) =2\cdot T(\frac{n}{2}) + \mathcal{O}(n) = ... = n\cdot T(1) + \sum_{i=0}^{log(n)} 2^i\cdot \mathcal{O}(\frac{n}{2^i}) =$$
$$ = \mathcal{O}(n) + \sum_{i=0}^{log(n)}  \mathcal{O}(n) = \mathcal{O}(n\cdot log(n)) $$

### quicksort algorithm

In [ ]:
# Sorting algorithm quicksort 

#  Function that check if x is under the pivot
def less_pivot(x, p):
    # The average of x is bigger than the pivot's one: so return false
    if x[1] > p[1]: return 0
    # The averages are the same, but the name of x comes first in the alphabeth: so return false
    if x[1] == p[1] and x[0] < p[0]: return 0
    # If I get to this point x is less than the pivot: return true
    return 1


def quicksort(average):
    if len(average) == 0 or len(average) == 1: return average
    # Get a random index (element) of the list
    p = random.randint(0, len(average)-1)
    A = []
    B = []
    # Divide the list in the ones under the pivot and the one over
    for i in range(len(average)):
        if i != p and less_pivot(average[i], average[p]):
            A.append(average[i])
        elif i != p:
            B.append(average[i])
    # Sort the smaller lists
    A = quicksort(A)
    B = quicksort(B)
    # Put everything together now that the lists are sorted 
    return B + [average[p]] + A

#### quicksort computational cost

The $quicksort$ function takes a list $l$, of length n, as input, then draw a random element from the list and define it as the pivot. The main thing the algoritghm does is devide the list in two smaller lists, one ($A$ of length $k$) where will be stored the elements of $l$ that are smaller than the pivot and the other ($B$ of length $n-k$) for the elements bigger than the pivot, then $quicksort$ call itself on the lists $A$ and $B$. 
$$ T(n) = T(k) + T(n-k) + \mathcal{O}(n) $$
We can then analyze the best and worst case:
- best for $k = \frac{n}{2}$:
$$ T(n) = 2T(\frac{n}{2})+\mathcal{O}(n) = ... = n\cdot T(1) + \sum_{i=0}^{log(n)} 2^i\cdot \mathcal{O}(\frac{n}{2^i}) = \mathcal{O}(n\cdot log(n)) $$
- worst with $k = 1$ all the times:
$$ T(n) = T(n-1) + \mathcal{O}(n) = ... = T(1) + \sum_{i=1}^{n}\mathcal{O}(i) =  \mathcal{O}(n^2)$$

### selectionsort algorithm

In [ ]:
# Sorting algorithm selectionsort 

# Function that check the max between x and y
def ismore(x, y):
    # The average of y is bigger: so I return false (0)
    if x[1] < y[1]: return 0
    # The average of y and x is equal, but the name of y comes first in the alphabeth: so I return false
    if x[1] == y[1] and x[0] > y[0]: return 0
    # If i get to this point x is the max
    return 1

def selectionsort (average):
    for i in range(len(average)):
        # Initialize the max
        max_i = average[i]
        idx = i
        # Scan the rest of the list to look for something bigger
        for j in range(i,len(average)):
            if ismore(average[j], max_i) and max_i != average[j]:
                # Save the new max and it's index
                max_i = average[j]
                idx = j
        # If the index of the max is not i, I change position between i and the max
        if idx != i:
            aux = average[i]
            average[i] = average[idx]
            average[idx] = aux
    return average

#### selectionsort computational cost

The $selectionsort$ is a straight foward algorithm, it looks for the maximum value and put it in the begining of the list, then move to the second bigger element and put it as the second element of the list, and so on. So talking about costs the $selectionsort$ we have:
$$T(n)=T(n-1)+\mathcal{O}(n)=...=\sum_{i=1}^{n}\mathcal{O}(i)=\mathcal{O}(\frac{n\cdot (n+1)}{2})=\mathcal{O}(n^2)$$ 

### Times

Let's actually try and time how much each of the sorting algorithms take to sort the ApplicantsInfo list

In [ ]:
# Function that print the results of the timing for all the sorting akgorithms
def showTimes(mergetime, quicktime, selecttime, applicants):
    print('Each method gave the same result, and we can take a look at sorted list. \nFirst the top 10 averages are:')
    for i in applicants[:10]:
        print('\t-'+str(i[0]), i[1])
    print('\nThe last 10 avereges are:')
    for i in applicants[-10:]:
        print('\t-'+str(i[0]), i[1])
    
    print('The time for each algorithm are the following:\n-', mergetime, 'for the mergesort.\n-', 
    quicktime, 'for the quicksort.\n-', selecttime, 'for the selectionsort.')
        

First thing we have to read the .txt file

In [ ]:
fileAppications = open('./ApplicantsInfo.txt', 'r')
Applications = fileAppications.read().splitlines()
del fileAppications

Now we can sort the results

In [ ]:
n, m = map(int, Applications[0].split())
applicants = []
for i in Applications[1:]:
    applicant = i.split()
    applicants.append((applicant[0]+' '+applicant[1], round(sum([int(applicant[x]) for x in range(2,len(applicant))])/m,2)))

#apply and time the mergesort
t1 = time.time()
MergeAppl = mergesort(applicants)
mergetime = time.time() - t1

#apply and time the quicksort
t1 = time.time()
QuickAppl = quicksort(applicants)
quicktime = time.time() - t1

#apply and time the selectionsort
t1 = time.time()
SelectAppl = selectionsort(applicants)
selecttime = time.time() - t1

And let's see if the results were what we expected and save the sorted list in a .txt file

In [ ]:
# Save the result for the sort
def saveResult(sorted):
    with open('RankingList.txt', 'w') as f:
        for applicant in sorted:
            row = applicant[0] + ' ' + str(applicant[1]) + '\n'   
            f.writelines(row)
    return

In [ ]:
# all the same so the sorting is good
if SelectAppl==QuickAppl and QuickAppl==MergeAppl:
    showTimes(mergetime, quicktime, selecttime, MergeAppl)
    saveResult(MergeAppl)

Each method gave the same result, and we can take a look at sorted list. 
First the top 10 averages are:
	-Emily Crispin 24.49
	-Patricia Witten 24.49
	-Bruce Johnson 24.45
	-Doreen Richmond 24.45
	-David Niederberger 24.44
	-Keisha Keene 24.44
	-Steven Boston 24.44
	-John Johnson 24.42
	-Marvin Ramirez 24.42
	-Melody Sanchez 24.42

The last 10 avereges are:
	-Helen Branch 23.58
	-Max Beck 23.58
	-Ray Bray 23.57
	-Eric Schmidt 23.56
	-Kenneth Marquez 23.56
	-Ruth Watts 23.56
	-Anna Martin 23.55
	-Justin Williams 23.55
	-Bruce Murphy 23.54
	-Mary Thompson 23.51
The time for each algorithm are the following:
- 0.31357765197753906 for the mergesort.
- 0.3477013111114502 for the quicksort.
- 193.54548239707947 for the selectionsort.


From the results is evident that the selectionsort algorithm is not the best when we are working with big numbers. 
    
In our case the length of the applicants list was $50000$ and, while both the quicksort and the mergesort are $\mathcal{O}(n\cdot log(n))$, the selectionsort is a $\mathcal{O}(n^2)$ and that makes a big difference since: 
    
- $n\cdot ln(n)$ with $n = 50000$ is: $5\cdot 10^5$
- $n^2$ with $n = 50000$ is: $2.5\cdot 10^9$